<a href="https://colab.research.google.com/github/Nero0113/se_code/blob/main/re_codebert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvidia-smi

Fri Aug  5 06:39:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

处理数据，生成codebert输入

In [ ]:
import os
import json
import numpy as np
from more_itertools import chunked
import random
import logging
from tqdm import tqdm, trange
logger = logging.getLogger(__name__)
from tensorboardX import SummaryWriter
DATA_DIR = '/content/drive/MyDrive/myclab/codebert/data'
def format_str(string):
  for char in ['\r\n', '\r', '\n', '\t']:
    string = string.replace(char, ' ')
  return string

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

class DataProcessor(object):
  """A single training/test example for simple sequence classification."""
  def __init__(self, pos_data, neg_data):

    self.pos_data = pos_data
    self.neg_data = neg_data

  def get_labels(self):

    return ["0", "1"]

  def get_train(self):
    # print("start get train")
    pos_len = len(self.pos_data)
    neg_len = len(self.neg_data)
    train_data = []
    for i in range(int(pos_len /100 * 70)):
      train_data.append(self.pos_data[i])
      train_data.append(self.neg_data[i])
    return train_data

  def get_val(self):
    # print("start get val")
    pos_len = len(self.pos_data)
    neg_len = len(self.neg_data)
    val_data = []
    val_data += self.pos_data[int(pos_len /100 * 70):int(pos_len /100 * 85)]
    val_data += self.neg_data[int(neg_len /100 * 70):int(neg_len /100 * 85)]
    return val_data

  def get_test(self):
    # print("start get test")
    pos_len = len(self.pos_data)
    neg_len = len(self.neg_data)
    test_data = []
    test_data += self.pos_data[int(pos_len /100 * 85):]
    test_data += self.neg_data[int(neg_len /100 * 85):]
    return test_data, test_data

  def convert_to_feature(self, examples, max_seq_len, tokenizer,
                        cls_token_at_end = False, pad_on_left = False, cls_token = '[CLS]',
                        sep_token = '[SEP]', pad_token = 0, sequence_a_segment_id=0,
                        sequence_b_segment_id=1,cls_token_segment_id=1, pad_token_segment_id=0,
                        mask_padding_with_zero=True):
    label_list = ["0", "1"]
    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    print("start convert to feature")
    for (ex_index, example) in enumerate(tqdm(examples)):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d" % (ex_index, len(examples)))

        tokens_a = tokenizer.tokenize(example['javadoc'])[:50]

        tokens_b = None
        if example['source']:
            tokens_b = tokenizer.tokenize(example['source'])
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_len - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_len - 2:
                tokens_a = tokens_a[:(max_seq_len - 2)]
        tokens = tokens_a + [sep_token]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if tokens_b:
            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

        if cls_token_at_end:
            tokens = tokens + [cls_token]
            segment_ids = segment_ids + [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)
        padding_length = max_seq_len - len(input_ids)

        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

        assert len(input_ids) == max_seq_len
        assert len(input_mask) == max_seq_len
        assert len(segment_ids) == max_seq_len

        label_id = label_map[str(example['label'])]

        if ex_index < 5:
            logger.info("*** Example ***")
            # logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (str(example['label']), label_id))

        features.append(
            InputFeatures(input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_id=label_id))
    return features

  
  

train_data:1675034\
val_data:358936\
test_data:358936

配置参数

In [ ]:
config = {
    "model_name_or_path": "$pretrained_model",
    "model_type": "roberta",
    "do_lower_case": True,
    "tokenizer_name": "uer/chinese_roberta_L-12_H-768",
    "model_name_or_path": "$pretrained_model",
    "max_seq_length": 256,
    "train_batch_size": 32,
    "eval_batch_size": 16,
    "gradient_accumulation_steps": 1,
    "learning_rate": 1e-5,
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "max_grad_norm": 1.0,
    "num_train_epochs": 4,
    "seed": 42,
    "data_dir": "/content/drive/MyDrive/myclab/codebert/test_data",
    "no_cuda": True,
    "output_dir": "/content/drive/MyDrive/myclab/codebert/models",
    "config_name": "",
    "start_step": "",
    "start_epoch": 0,
    "adam_epsilon": 1e-8,
    "weight_decay": 0.0,
    "max_steps": -1,
    "gradient_accumulation_steps": 1,
    "warmup_steps": 0,
    "logging_steps": 50,
    "evaluate_during_training": False,
    "local_rank": -1,
    "task_name": "codesearch", 
    "do_eval": True,
    "pred_model_dir": "/content/drive/MyDrive/myclab/codebert/models/checkpoint-best",
    "test_result_dir": "/content/drive/MyDrive/myclab/codebert/result/${idx}_batch_result.txt",
    "eval_all_checkpoints": True,

}



In [ ]:
import json
import os
import argparse
import random
import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from transformers import (WEIGHTS_NAME, get_linear_schedule_with_warmup, AdamW,
                          RobertaConfig,
                          RobertaForSequenceClassification,
                          RobertaTokenizer,AutoTokenizer,
                          AutoModelForMaskedLM)
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaForSequenceClassification, AutoTokenizer)}


def set_seed(args):
    random.seed(config["seed"])
    np.random.seed(config["seed"])
    torch.manual_seed(config["seed"])

def load_and_cache_examples(config, processor, tokenizer, ttype='train'):
    # if ttype == 'train':
    #     file_name = args.train_file.split('.')[0]
    # elif ttype == 'dev':
    #     file_name = args.dev_file.split('.')[0]
    # elif ttype == 'test':
    #     file_name = args.test_file.split('.')[0]
    # print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    cached_features_file = os.path.join(config['data_dir'], 'cached_{}_{}_{}'.format(
        ttype,
        list(filter(None, config['model_name_or_path'].split('/'))).pop(),
        str(config["max_seq_length"])))

    try:
        features = torch.load(cached_features_file)
        if ttype == 'test':
          examples, instances = processor.get_test()
    except:
        label_list = processor.get_labels()
        if ttype == 'train':
            examples = processor.get_train()
        elif ttype == 'dev':
            examples = processor.get_val()
        elif ttype == 'test':
            examples, instances = processor.get_test()

        features = processor.convert_to_feature(examples, config["max_seq_length"], tokenizer)
    
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

    # print(all_input_ids.shape)
    # print(all_input_mask.shape)
    # print(all_segment_ids.shape)
    # print(all_label_ids.shape)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    
    if (ttype == 'test'):
        return dataset, instances
    else:
        return dataset



    

In [ ]:
pos_data = []
neg_data = []
for i in range(1):
    # pos_path_str = '/content/drive/MyDrive/myclab/codebert/data/data_' + str(i) + '.json'
    # neg_path_str = '/content/drive/MyDrive/myclab/codebert/data/negative_data_' + str(i) + '.json'
    pos_path_str = '/content/drive/MyDrive/myclab/codebert/data/mixed_pos.json'
    neg_path_str = '/content/drive/MyDrive/myclab/codebert/data/mixed_neg.json'
    pos_data += json.load(open(pos_path_str, 'r', encoding='utf-8'))
    neg_data += json.load(open(neg_path_str, 'r', encoding='utf-8'))

processer = DataProcessor(pos_data, neg_data)
def train(args, train_dataset, model, tokenizer, optimizer):

  if config["local_rank"] in [-1, 0]:
    tb_writer = SummaryWriter()

  train_sampler = RandomSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=config["train_batch_size"])

  if config["max_steps"] > 0:
    t_total = config["max_steps"]
    config["num_train_epochs"] = config["max_steps"] // (len(train_dataloader) // config["gradient_accumulation_steps"]) + 1
  else:
    t_total = len(train_dataloader) // config["gradient_accumulation_steps"] * config["num_train_epochs"]

  scheduler = get_linear_schedule_with_warmup(optimizer, config["warmup_steps"], t_total)

  checkpoint_last = os.path.join(config["output_dir"], 'checkpoint-last')
  scheduler_last = os.path.join(checkpoint_last, 'scheduler.pt')
  if os.path.exists(scheduler_last):
      scheduler.load_state_dict(torch.load(scheduler_last))
  
  logger.info("***** Running training *****")
  logger.info("  Num examples = %d", len(train_dataset))
  logger.info("  Num Epochs = %d", config["num_train_epochs"])
  logger.info("  Instantaneous batch size = %d", config["train_batch_size"])
  # logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
  #             config["train_batch_size"] * config["gradient_accumulation_steps"] * (
  #                 torch.distributed.get_world_size()))
  logger.info("  Gradient Accumulation steps = %d", config["gradient_accumulation_steps"])
  logger.info("  Total optimization steps = %d", t_total)

  global_step = config["start_step"]
  tr_loss, logging_loss = 0.0, 0.0
  best_acc = 0.0
  model.zero_grad()

  train_iterator = trange(config["start_epoch"], int(config["num_train_epochs"]), desc="Epoch")
  set_seed(args)
  model.train()
  for idx, _ in enumerate(train_iterator):
    tr_loss = 0.0
    for step, batch in enumerate(train_dataloader):
      # print(len(batch))
      batch = tuple(t.to(device) for t in batch)
      inputs = {'input_ids': batch[0],
            'attention_mask': batch[1],
            # 'token_type_ids': batch[2] if args.model_type in ['bert', 'xlnet'] else None,
            # # XLM don't use segment_ids
            'labels': batch[3]}
      ouputs = model(**inputs)
      loss = ouputs[0]
      print("loss: ",loss)
      #loss = ouputs[0]

      if config["gradient_accumulation_steps"] > 1:
        loss = loss / config["gradient_accumulation_steps"]
      
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), config["max_grad_norm"])

      tr_loss += loss.item()
      if (step + 1) % config["gradient_accumulation_steps"] == 0:
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()
        global_step += 1

        if config["local_rank"] in [-1, 0] and config["logging_steps"] > 0 and global_step % config["logging_steps"] == 0:
          if config["evaluate_during_training"]:
              results = evaluate(args, model, tokenizer, checkpoint=str(global_step))
              for key, value in results.items():
                # tb_writer.add_scalar('eval_{}'.format(key), value, global_step)
                logger.info('loss %s', str(tr_loss - logging_loss))
          # tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
          # tb_writer.add_scalar('loss', (tr_loss - logging_loss) / args.logging_steps, global_step)
          logging_loss = tr_loss

      if config["max_steps"] > 0 and global_step > config["max_steps"]:
        # epoch_iterator.close()
        break

    if config["do_eval"]:
      results = evaluate(args, model, tokenizer, checkpoint=str(config["start_epoch"] + idx))

      last_output_dir = os.path.join(config["output_dir"], 'checkpoint-last')
      if not os.path.exists(last_output_dir):
        os.makedirs(last_output_dir)
      model_to_save = model.module if hasattr(model,'module') else model
      model_to_save.save_pretrained(last_output_dir)
      logger.info("Saving model checkpoint to %s", last_output_dir)
      idx_file = os.path.join(last_output_dir, 'idx_file.txt')
      with open(idx_file, 'w', encoding='utf-8') as idxf:
        idxf.write(str(config["start_epoch"] + idx) + '\n')

      torch.save(optimizer.state_dict(), os.path.join(last_output_dir, "optimizer.pt"))
      torch.save(scheduler.state_dict(), os.path.join(last_output_dir, "scheduler.pt"))
      logger.info("Saving optimizer and scheduler states to %s", last_output_dir)

      step_file = os.path.join(last_output_dir, 'step_file.txt')
      with open(step_file, 'w', encoding='utf-8') as stepf:
        stepf.write(str(global_step) + '\n')

      if (results['acc'] > best_acc):
        best_acc = results['acc']
        output_dir = os.path.join(config["output_dir"], 'checkpoint-best')
        if not os.path.exists(output_dir):
          os.makedirs(output_dir)
          model_to_save = model.module if hasattr(model,'module') else model
          model_to_save.save_pretrained(output_dir)
          torch.save(args, os.path.join(output_dir, 'training_{}.bin'.format(idx)))
          logger.info("Saving model checkpoint to %s", output_dir)

          torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
          torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
          logger.info("Saving optimizer and scheduler states to %s", output_dir)
    
    if config["max_steps"] > 0 and global_step > config["max_steps"]:
      train_iterator.close()
      break

  
  return global_step, tr_loss / global_step


In [ ]:
from sklearn.metrics import f1_score
def accuracy(out, labels):
  outputs = np.argmax(out, axis=1)
  return np.sum(outputs == labels)

def simple_accuracy(preds, labels):
    return (preds == labels).mean()

def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }

def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    if task_name == "codesearch":
        return acc_and_f1(preds, labels)
    else:
        raise KeyError(task_name)

def evaluate(args, model, tokenizer, checkpoint=None, prefix="", mode='dev'):
  eval_outputs_dirs = (config["output_dir"],)
  eval_task_names = (config["task_name"],)
  results = {}
  for eval_task, eval_output_dir in zip(eval_task_names, eval_outputs_dirs):
    if (mode == 'dev'):
      eval_dataset = load_and_cache_examples(args, processer, tokenizer, ttype='dev')
    elif (mode == 'test'):
      eval_dataset, instances = load_and_cache_examples(args, processer, tokenizer, ttype='test')
    
    if not os.path.exists(eval_output_dir):
      os.makedirs(eval_output_dir)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=config["eval_batch_size"])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", config["eval_batch_size"])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
      model.eval()
      batch = tuple(t.to(device) for t in batch)

      with torch.no_grad():
        inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              # 'token_type_ids': batch[2],
              # XLM don't use segment_ids
              'labels': batch[3]}

        outputs = model(**inputs)
        tmp_eval_loss, logits = outputs[:2]

        eval_loss += tmp_eval_loss.mean().item()
      
      nb_eval_steps += 1
      if preds is None:
        preds = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
      else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

    preds_label = np.argmax(preds, axis=1)
    result = compute_metrics(eval_task, preds_label, out_label_ids)
    results.update(result)
    if (mode == 'dev'):
      output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
      with open(output_eval_file, "a+") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        writer.write('evaluate %s\n' % checkpoint)
        for key in sorted(result.keys()):
          logger.info("  %s = %s", key, str(result[key]))
          writer.write("%s = %s\n" % (key, str(result[key])))
    elif (mode == 'test'):
      output_test_file = config["test_result_dir"]
      output_dir = os.path.dirname(output_test_file)
      if not os.path.exists(output_dir):
        os.makedirs(output_dir)
      with open(output_test_file, "w") as writer:
        logger.info("***** Output test results *****")
        all_logits = preds.tolist()
        for i, logit in tqdm(enumerate(all_logits), desc='Testing'):
          instance_rep = '<CODESPLIT>'.join(
            [item.encode('utf-8', 'ignore').decode('utf-8') for item in instances[i]])
        
          writer.write(instance_rep + '<CODESPLIT>' + '<CODESPLIT>'.join([str(l) for l in logit]) + '\n')
        
        for key in sorted(result.keys()):
          print("%s = %s" % (key, str(result[key])))

  return results


主程序

In [ ]:
args = config
import glob
if args["no_cuda"]:
    device = torch.device("cuda" if torch.cuda.is_available() and not args["no_cuda"] else "cpu")

set_seed(args)



label_list = processer.get_labels()
num_labels = len(label_list)

config["start_epoch"] = 0
config["start_step"] = 0

checkpoint_last = os.path.join(args["output_dir"], 'checkpoint-last')
if os.path.exists(checkpoint_last) and os.listdir(checkpoint_last):
    config['model_name_or_path'] = os.path.join(checkpoint_last, 'pytorch_model.bin')
    config["config_name"] = os.path.join(checkpoint_last, 'config.json')
    idx_file = os.path.join(checkpoint_last, 'idx_file.txt')
    with open(idx_file, encoding='utf-8') as idxf:
        config["start_epoch"] = int(idxf.readlines()[0].strip()) + 1

    step_file = os.path.join(checkpoint_last, 'step_file.txt')
    if os.path.exists(step_file):
        with open(step_file, encoding='utf-8') as stepf:
            config["start_step"] = int(stepf.readlines()[0].strip())

config_class, model_class, tokenizer_class = MODEL_CLASSES[config["model_type"]]
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=config["do_lower_case"])
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

model.to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay': config["weight_decay"]},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
optimizer = AdamW(optimizer_grouped_parameters, lr=config["learning_rate"], eps=config["adam_epsilon"])

optimizer_last = os.path.join(checkpoint_last, 'optimizer.pt')
if os.path.exists(optimizer_last):
    optimizer.load_state_dict(torch.load(optimizer_last))

train_dataset = load_and_cache_examples(args, processer, tokenizer, ttype='train')
global_step, tr_loss = train(args, train_dataset, model, tokenizer, optimizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

if not os.path.exists(config["output_dir"]) and config["local_rank"] in [-1, 0]:
  os.makedirs(config["output_dir"])
logger.info("Saving model checkpoint to %s", config["output_dir"])
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(config["output_dir"])
tokenizer.save_pretrained(config["output_dir"])
torch.save(args, os.path.join(config["output_dir"], 'training_args.bin'))
model = model_class.from_pretrained(config["output_dir"])
tokenizer = tokenizer_class.from_pretrained(config["output_dir"])
model.to(device)

results = {}
checkpoints = [config["output_dir"]]
if config["eval_all_checkpoints"]:
  checkpoints = list(
    os.path.dirname(c) for c in sorted(glob.glob(config["output_dir"] + '/**/' + WEIGHTS_NAME, recursive=True)))
  logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  #
logger.info("Evaluate the following checkpoints: %s", checkpoints)
for checkpoint in checkpoints:
  print(checkpoint)
  global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
  model = model_class.from_pretrained(checkpoint)
  model.to(device)
  result = evaluate(args, model, tokenizer, checkpoint=checkpoint, prefix=global_step)
  result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
  results.update(result)

print('testing')
model = model_class.from_pretrained(config["pred_model_dir"])
model.to(device)
evaluate(args, model, tokenizer, checkpoint=None, prefix='', mode='test')


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

start convert to feature


100%|██████████| 1674/1674 [00:04<00:00, 410.80it/s]
Epoch: 0it [00:00, ?it/s]


/content/drive/MyDrive/myclab/codebert/models/checkpoint-best
start convert to feature


Evaluating: 100%|██████████| 23/23 [02:02<00:00,  5.31s/it]


...

Evaluating: 100%|██████████| 23/23 [02:00<00:00,  5.24s/it]


testing
start convert to feature


Evaluating: 100%|██████████| 23/23 [02:01<00:00,  5.27s/it]
Testing: 360it [00:00, 196864.33it/s]

acc = 0.49166666666666664
acc_and_f1 = 0.5523872445384073
f1 = 0.6131078224101479


{'acc': 0.49166666666666664,
 'f1': 0.6131078224101479,
 'acc_and_f1': 0.5523872445384073}